# fb testing

In [1]:
import numpy as np
import os
import pickle
import dask as pd
import pandas as pd_old
from datetime import datetime
import warnings
import utils

from feature_extraction import SubjectData, compute_features, get_samples, combine_files



In [4]:

# To ignore all warnings:
warnings.filterwarnings("ignore", module="numpy")

WINDOW_IN_SECONDS = 60
stride = 1
label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0}
int_to_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
feat_names = None
loadPath = '../data/WESAD'
savePath = '../data/GN-WESAD'
subject_feature_path = '/subject_feats'
#rootdir = 'data/WESAD'

subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
#snrs = [0.0001, 0.00001, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6]
snrs = [0.1]
n_samples = 10 # Number of samples to generate for each SNR
    

if not os.path.exists(savePath):
    os.makedirs(savePath)
if not os.path.exists(savePath + subject_feature_path):
    os.makedirs(savePath + subject_feature_path)


In [8]:

# Make directories for each SNR and each sample
for n_i in range(n_samples):
    for snr in snrs:
        if not os.path.exists(savePath + '/n_'+str(n_i)+'/snr_'+str(snr)+ subject_feature_path):
            os.makedirs(savePath + '/n_'+str(n_i)+'/snr_'+str(snr)+ subject_feature_path)
        for subject_id in subject_ids:
            if not os.path.exists(savePath + '/n_'+str(n_i)+'/snr_'+str(snr)+ '/S'+str(subject_id)):
                os.makedirs(savePath + '/n_'+str(n_i)+'/snr_'+str(snr)+ '/S'+str(subject_id))
subject = SubjectData(main_path='data/WESAD', subject_number=2)
data_dict = subject.get_wrist_and_chest_data()

In [47]:
os.listdir('../mnt/box /')

FileNotFoundError: [Errno 2] No such file or directory: '../mnt/box /'